In [1]:
"""
Adapted from keras example cifar10_cnn.py
Train ResNet-18 on the CIFAR10 small images dataset.

GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10.py
"""
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from keras import optimizers

import numpy as np
import pandas
import resnet
from PIL import Image
from skimage import io
import os
import sys

Using Theano backend.
1 #define _CUDA_NDARRAY_C
2 
3 #include <Python.h>
4 #include <structmember.h>
5 #include "theano_mod_helper.h"
6 
7 #include <numpy/arrayobject.h>
8 #include <iostream>
9 
10 #include "cuda_ndarray.cuh"
11 
12 #ifndef CNMEM_DLLEXPORT
13 #define CNMEM_DLLEXPORT
14 #endif
15 
16 #include "cnmem.h"
17 #include "cnmem.cpp"
18 
19 //If true, when there is a gpu malloc or free error, we print the size of allocated memory on the device.
20 #define COMPUTE_GPU_MEM_USED 0
21 
22 //If true, we fill with NAN allocated device memory.
23 #define ALLOC_MEMSET 0
24 
25 //If true, we print out when we free a device pointer, uninitialize a
26 //CudaNdarray, or allocate a device pointer
27 #define PRINT_FREE_MALLOC 0
28 
29 //If true, we do error checking at the start of functions, to make sure there
30 //is not a pre-existing error when the function is called.
31 //You probably need to set the environment variable
32 //CUDA_LAUNCH_BLOCKING=1, and/or modify the CNDA_THREAD_SYNC
33

1310                         CudaNdarray_HOST_STRIDES(out)[2],
1311                         CudaNdarray_DEV_DATA(self),
1312                         CudaNdarray_HOST_DIMS(self)[0], //For indices check
1313                         CudaNdarray_HOST_STRIDES(self)[0], //strides
1314                         CudaNdarray_HOST_STRIDES(self)[1],
1315                         CudaNdarray_HOST_STRIDES(self)[2],
1316                         err_var);
1317             }
1318             break;
1319     default:
1320         PyErr_SetString(PyExc_NotImplementedError,
1321                         "CudaNdarray_TakeFrom: only input with 1, 2 or 3"
1322                         " dimensions are currently supported");
1323 
1324     }
1325     free(dims);
1326     CNDA_THREAD_SYNC;
1327     cudaError_t err = cudaGetLastError();
1328     if (cudaSuccess != err) {
1329         PyErr_Format(PyExc_RuntimeError,
1330                      "Cuda error: %s: %s.\n",
1331                      "CudaNdarray_TakeFrom",

3053     {NULL, NULL, NULL, NULL}  /* Sentinel */
3054 };
3055 
3056 PyObject *CudaNdarray_repr(PyObject *self)
3057 {
3058     CudaNdarray *object = (CudaNdarray *)self;
3059     PyObject * np_object = CudaNdarray_CreateArrayObj(object);
3060     PyObject * str = PyObject_Str((PyObject *) np_object);
3061     char * cstr = PyString_AsString(str);
3062     PyObject * out = PyString_FromFormat("%s%s%s",
3063                         "CudaNdarray(",
3064                         cstr,
3065                         ")");
3066     Py_DECREF(str);
3067     Py_DECREF(np_object);
3068     #if PY_MAJOR_VERSION >= 3
3069     // In Python 3 PyString_FromFormat return a Bytes object
3070     PyObject* out2 = PyObject_Str(out);
3071     Py_DECREF(out);
3072     return out2;
3073     #endif
3074     return out;
3075 }
3076 
3077 static PyTypeObject CudaNdarrayType =
3078 {
3079 #if PY_MAJOR_VERSION >= 3
3080     PyVarObject_HEAD_INIT(NULL, 0)
3081 #else
3082     PyObject_HEAD_INIT(NULL)
3083     0,   

4944         {
4945             newdims[i] = 1;
4946             newstrides[i] = 0;
4947         }
4948         else if(dims_taken[pattern[i]])
4949         {
4950             PyErr_Format(PyExc_ValueError, "Cudandarray_dimshuffle: invalid pattern for Cudandarray_dimshuffle. You used the dimensions %d multiple time",
4951                          pattern[i]);
4952             free(newdims);
4953             return -1;
4954         }
4955         else if (pattern[i]>= self->nd)
4956         {
4957             PyErr_Format(PyExc_ValueError, "Cudandarray_dimshuffle: invalid pattern for Cudandarray_dimshuffle. You asked for a dimensions that don't exist %d for a %d dims CudaNdarray",
4958                          pattern[i], self->nd);
4959             free(newdims);
4960             return -1;
4961         }
4962         else
4963         {
4964             newdims[i] = CudaNdarray_HOST_DIMS(self)[pattern[i]];
4965             newstrides[i] = CudaNdarray_HOST_STRIDES(self)[pattern[i]];
49

nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc fatal   : Cannot find compiler 'cl.exe' in PATH

['nvcc', '-shared', '-O3', '-Xlinker', '/DEBUG', '-D HAVE_ROUND', '-m64', '-Xcompiler', '-DCUDA_NDARRAY_CUH=mc72d035fdf91890f3b36710688069b2e,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,/Zi,/MD', '-I"C:\\Users\\VIPLAB\\AppData\\Local\\conda\\conda\\envs\\keras\\lib\\site-packages\\theano\\sandbox\\cuda"', '-I"C:\\Users\\VIPLAB\\AppData\\Local\\conda\\conda\\envs\\keras\\lib\\site-packages\\numpy\\core\\include"', '-I"C:\\Users\\VIPLAB\\AppData\\Local\\conda\\conda\\envs\\keras\\include"', '-I"C:\\Users\\VIPLAB\\AppData\\Local\\conda\\conda\\envs\\keras\\lib\\site-packages\\theano\\gof"', '-L"C:\\Users\\VIPLAB\\AppData\\Local\\conda\\conda\\envs\\keras\\libs"', '-L"C:\\Users\\VIPLAB\\AppData\\Local\\conda\\conda\\envs\\keras"', '-o', 'C:\\Users\\VIPLAB\\AppDa

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'C:\\Users\\VIPLAB\\AppData\\Local\\Temp\\try_flags_q8kynl_b.c:4:19: fatal error: cudnn.h: No such file or directory\r\ncompilation terminated.\r\n'
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:01:00.0)


# load data

In [2]:
# data dir
directory = "./lfw_only_face_resize/"
train_x = []
train_y = []
test_x = []
test_y = []
threhold = 3
count = 0
for _dir in os.listdir(directory):
#     只訓練>5張的dataset
    people_image_count = len(os.listdir(os.path.join(directory, _dir)))
    if(people_image_count >= threhold):
        count += 1
        for index, filename in enumerate(os.listdir(os.path.join(directory, _dir))):
            if filename.endswith(".jpg"):
                img_path = os.path.join(directory, _dir, filename)
                image = Image.open(img_path)
#                 每個人取1張當作testing
                if(index == 0):
                    test_x.append(np.array(image))
                    test_y.append(_dir)
#                     break
                else:
                    train_x.append(np.array(image))
                    train_y.append(_dir)

In [3]:
count

901

In [4]:
train_x, train_y = np.array(train_x), np.array(train_y)
test_x, test_y_name = np.array(test_x), np.array(test_y)
train_x, test_x = train_x.astype('float32'), test_x.astype('float32')
train_y, test_y = pandas.get_dummies(train_y).values, pandas.get_dummies(test_y_name).values

In [5]:
train_x /= 255
test_x /= 255

# model setting

In [6]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=20)
csv_logger = CSVLogger('resnet64_faceValidation.csv')
# filepath="weights-improvement-{epoch:06d}-{val_acc:.4f}.hdf5"
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint("./model_save/" + filepath , monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# classes = os.listdir(directory)

In [7]:
batch_size = 128
nb_classes = count
nb_epoch = 100000
data_augmentation = False


In [8]:
# input image dimensions
img_rows, img_cols = 104, 96
# The CIFAR10 images are RGB.
img_channels = 3

In [9]:
# def load_trained_model(weights_path):
#     model = create_model()
#     model.load_weights(weights_path)
#    

In [10]:
# model = resnet.ResnetBuilder.build_resnet_64((img_channels, img_rows, img_cols), nb_classes)
model = resnet.ResnetBuilder.build_resnet_34((img_channels, img_rows, img_cols), nb_classes)
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd = optimizers.SGD(lr=0.1, decay=5e-4, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [11]:
train_x.shape

(6705, 104, 96, 3)

In [12]:
train_y.shape

(6705, 901)

In [13]:
model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=nb_epoch,
          shuffle=True,
          validation_split=0.2,
          verbose=1,
#           callbacks=[lr_reducer, early_stopper, csv_logger, checkpoint])
          callbacks=[lr_reducer, csv_logger, checkpoint])


Train on 5364 samples, validate on 1341 samples
Epoch 1/100000
5364/5364 [==============================] - 167s 31ms/step - loss: 7.2576 - acc: 0.1217 - val_loss: 13.0987 - val_acc: 0.0000e+00
Epoch 2/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 6.2020 - acc: 0.2340 - val_loss: 12.6443 - val_acc: 0.0000e+00
Epoch 3/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 5.2857 - acc: 0.3538 - val_loss: 10.7900 - val_acc: 0.0000e+00
Epoch 4/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 4.3768 - acc: 0.4802 - val_loss: 11.5050 - val_acc: 0.0000e+00
Epoch 5/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 3.4704 - acc: 0.6188 - val_loss: 12.4563 - val_acc: 0.0000e+00
Epoch 6/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 2.6427 - acc: 0.7858 - val_loss: 13.9657 - val_acc: 0.0000e+00
Epoch 7/100000
5364/5364 [==============================] - 166s 31ms/step -

Epoch 31/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3660 - acc: 1.0000 - val_loss: 14.7431 - val_acc: 0.0052
Epoch 32/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3660 - acc: 1.0000 - val_loss: 14.7436 - val_acc: 0.0052
Epoch 33/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7448 - val_acc: 0.0052
Epoch 34/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7448 - val_acc: 0.0052
Epoch 35/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7458 - val_acc: 0.0052
Epoch 36/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7458 - val_acc: 0.0052
Epoch 37/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7466 - val_acc: 0.0052

Epoch 63/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7454 - val_acc: 0.0052
Epoch 64/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7455 - val_acc: 0.0052
Epoch 65/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7454 - val_acc: 0.0052
Epoch 66/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7452 - val_acc: 0.0052
Epoch 67/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3657 - acc: 1.0000 - val_loss: 14.7452 - val_acc: 0.0052
Epoch 68/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7457 - val_acc: 0.0052
Epoch 69/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7455 - val_acc: 0.0052

Epoch 95/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7443 - val_acc: 0.0052
Epoch 96/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7447 - val_acc: 0.0052
Epoch 97/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7454 - val_acc: 0.0052
Epoch 98/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7461 - val_acc: 0.0052
Epoch 99/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7457 - val_acc: 0.0052
Epoch 100/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7443 - val_acc: 0.0052
Epoch 101/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7447 - val_acc: 0.00

5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7446 - val_acc: 0.0052
Epoch 127/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7447 - val_acc: 0.0052
Epoch 128/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7465 - val_acc: 0.0052
Epoch 129/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7457 - val_acc: 0.0052
Epoch 130/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7452 - val_acc: 0.0052
Epoch 131/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7458 - val_acc: 0.0052
Epoch 132/100000
5364/5364 [==============================] - 165s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7451 - val_acc: 0.0052
Epoch 133

...

5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7463 - val_acc: 0.0052
Epoch 2204/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7452 - val_acc: 0.0052
Epoch 2205/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7452 - val_acc: 0.0052
Epoch 2206/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7439 - val_acc: 0.0052
Epoch 2207/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7454 - val_acc: 0.0052
Epoch 2208/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7462 - val_acc: 0.0052
Epoch 2209/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7441 - val_acc: 0.0052
Epo

5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7449 - val_acc: 0.0052
Epoch 2235/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7450 - val_acc: 0.0052
Epoch 2236/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7440 - val_acc: 0.0052
Epoch 2237/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7438 - val_acc: 0.0052
Epoch 2238/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3657 - acc: 1.0000 - val_loss: 14.7446 - val_acc: 0.0052
Epoch 2239/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3657 - acc: 1.0000 - val_loss: 14.7436 - val_acc: 0.0052
Epoch 2240/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7434 - val_acc: 0.0052
Epo

5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7458 - val_acc: 0.0052
Epoch 2266/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7438 - val_acc: 0.0052
Epoch 2267/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7439 - val_acc: 0.0052
Epoch 2268/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7444 - val_acc: 0.0052
Epoch 2269/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3657 - acc: 1.0000 - val_loss: 14.7442 - val_acc: 0.0052
Epoch 2270/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7438 - val_acc: 0.0052
Epoch 2271/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3659 - acc: 1.0000 - val_loss: 14.7459 - val_acc: 0.0052
Epo

5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7436 - val_acc: 0.0052
Epoch 2297/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7444 - val_acc: 0.0052
Epoch 2298/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7448 - val_acc: 0.0052
Epoch 2299/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7441 - val_acc: 0.0052
Epoch 2300/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3657 - acc: 1.0000 - val_loss: 14.7445 - val_acc: 0.0052
Epoch 2301/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7464 - val_acc: 0.0052
Epoch 2302/100000
5364/5364 [==============================] - 166s 31ms/step - loss: 1.3658 - acc: 1.0000 - val_loss: 14.7461 - val_acc: 0.0052
Epo

KeyboardInterrupt: 

In [14]:
# save
output_h5 = "sgd_resnet34_my_final_balance_model_%d.h5" % threhold
model.save(output_h5)   # HDF5 file, you have to pip3 install h5py if don't have it

In [15]:
# print('test before save: ', model.predict(X_test[0:2]))


In [16]:
score = model.evaluate(test_x, test_y, batch_size=128)

901/901 [==============================] - 8s 9ms/step


In [17]:
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

acc: 18.31%
